# Libraries imported

In [11]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlalchemy
from google.colab import userdata
from pathlib import Path
from sqlalchemy import create_engine

# Data dowload process and database conection.


In [19]:
# 1.Data load process, API Kaggle, Neon conection test

def data_load_connection():
  """
  This function automatically dowload the dataset from Kaggle and connect to Neon.tech.
  """
        # 1.1 API de Kaggle
  os.environ['KAGGLE_USERNAME'] = userdata.get('KAGGLE_USERNAME')
  os.environ['KAGGLE_KEY'] = userdata.get('KAGGLE_KEY')

      # 1.2 Data dowload
  DATA_PATH = "/content/drive/MyDrive/[03]_proyectos_/GitHub/telco_customers_project/data"

  !kaggle datasets download -d blastchar/telco-customer-churn -p {DATA_PATH} --unzip

  print(f"✅ Dowload completed {DATA_PATH}")

    # 1.3. Neon conection test
  DB_URL = userdata.get('DB_URL')

  try:
      engine = sqlalchemy.create_engine(DB_URL)
      with engine.connect() as conn:
       print("Connection successful")
  except Exception as e:
      print(f"Conection error: {e}")

# Data preprocessing

In [4]:
# Data load and preprocessing

def data_load_preprocessing (base_path, columns_numeric):

  """
  It automatically searches for the first CSV in a directory, loads it, normalizes column names,
  and converts specific data types.
  """

  # Base path
  base_path_ = Path(base_path)

  # Checking for .csv files
  csv_files = list(base_path_.glob('*.csv'))
  if not csv_files:
    print("No .csv files in the directory")
    return None
  else:
   target_file = csv_files[0]
   print(f"File detected: {target_file.name}\n")

  # Data load
  df = pd.read_csv(target_file)

  # Column's name to lower and blak space replace with "_"
  df.columns = [c.lower().replace(" ", "_")for c in df.columns]

  # Column's data type changed tu numeric
  for col in columns_numeric:
    df[col] = pd.to_numeric(df[col], errors="coerce")
  print(f"{df.info()}\n Data description (only numerics):\n{df.describe()}\n Null data to be addressed\n{df.isnull().sum()}\n")

  # Columns filled with 0

  null_columns = df.columns[df.isnull().any()].to_list()
  if null_columns:
    df = df.fillna(0)
    print(f"Columns with nulls addressed {null_columns}")
  else:
    print("No columns with nulls")

  return df




# Cleaned data uploaded  to the database.

In [5]:
# Data load to Neon.Tech
def upload_data_to_neon(df, table_name):
  """Upload dataframe to Neon.tech, only requiers dataframe and table_name"""

  # Trye conection with db
  try:
    db_url = userdata.get('DB_URL')
    if not db_url:
      print("DB_URL environment variable not set")
      return

    # Upload df
    engine = sqlalchemy.create_engine(db_url)
    df.to_sql(table_name, engine, if_exists='replace', index=False)
    print(f"💻 Data uploaded to table: {table_name} ")
  except Exception as e:
    print(f"Error: {e} ❌")


# Automated EDA report.

In [21]:

def run_sql_eda():
  """
  Automatically performe a EDA report based in three queries:
  1) Global churn rate
  2) Churn by contract type
  3) Financial risk
  """
    # 1. Conexión (Asegúrate de que 'DB_URL' esté en tus Secrets)
  engine = create_engine(userdata.get('DB_URL'))

    # 2. Diccionario de consultas (Corregido: 'queries' y sintaxis SQL)
  queries = {
        "Global churn rate": """
            SELECT
                COUNT(*) AS total_customers,
                SUM(CASE WHEN churn = 'Yes' THEN 1 ELSE 0 END) AS total_churned,
                ROUND(SUM(CASE WHEN churn = 'Yes' THEN 1 ELSE 0 END)*100.0 / COUNT(*), 2) AS churn_rate_percentage
            FROM churn_data_cleaned;
        """,
        "Churn by contract type": """
            SELECT
                contract,
                ROUND(AVG(monthlycharges)::numeric, 2) AS avg_charges,
                ROUND(SUM(CASE WHEN churn = 'Yes' THEN 1 ELSE 0 END)*100.0 / COUNT(*), 2) AS churn_rate_percentage
            FROM churn_data_cleaned
            GROUP BY contract
            ORDER BY churn_rate_percentage ASC;
        """,
        "Financial risk": """
            WITH financial_risk AS (
                SELECT
                    CASE
                        WHEN monthlycharges <= 30 THEN 'Low (0-30)'
                        WHEN monthlycharges > 30 AND monthlycharges <= 70 THEN 'Medium (31-70)'
                        ELSE 'High (70+)'
                    END AS risk_category,
                    COUNT(*) AS total_customers,
                    SUM(CASE WHEN churn = 'Yes' THEN 1 ELSE 0 END) AS total_churned,
                    ROUND(SUM(CASE WHEN churn = 'Yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS churn_rate_percentage
                FROM churn_data_cleaned
                GROUP BY risk_category
            )
            SELECT * FROM financial_risk
            ORDER BY churn_rate_percentage DESC;
        """
    }

    # 3. El reporte (Ahora DENTRO de la función para que tenga acceso a 'queries' y 'engine')
  print("📊 --- SQL EDA REPORT --- 📊\n")
  for title, query_sql in queries.items():
       print(f"🔹 Query title: {title}")
       try:
          temp_df = pd.read_sql(query_sql, engine)
          display(temp_df)
       except Exception as e:
        print(f"Error en esta consulta: {e}")
        print("-" * 30 + "\n")


In [7]:
upload_data_to_neon(df, "churn_data_cleaned")

💻 Data uploaded to table: churn_data_cleaned 


In [6]:
df= data_load_preprocessing("/content/drive/MyDrive/[03]_proyectos_/GitHub/telco_customers_project/data", ["monthlycharges", "totalcharges"])

File detected: WA_Fn-UseC_-Telco-Customer-Churn.csv

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerid        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   seniorcitizen     7043 non-null   int64  
 3   partner           7043 non-null   object 
 4   dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   phoneservice      7043 non-null   object 
 7   multiplelines     7043 non-null   object 
 8   internetservice   7043 non-null   object 
 9   onlinesecurity    7043 non-null   object 
 10  onlinebackup      7043 non-null   object 
 11  deviceprotection  7043 non-null   object 
 12  techsupport       7043 non-null   object 
 13  streamingtv       7043 non-null   object 
 14  streamingmovies   7043 non-null   object 
 15  contract          7043 non-null   ob

In [17]:
run_sql_eda()

📊 --- SQL EDA REPORT --- 📊

🔹 Query title: Global churn rate


,total_customers,total_churned,churn_rate_percentage
0,7043,1869,26.54


------------------------------

🔹 Query title: Churn by contract type


,contract,avg_charges,churn_rate_percentage
0,Two year,60.77,2.83
1,One year,65.05,11.27
2,Month-to-month,66.40,42.71


------------------------------

🔹 Query title: Financial risk


,risk_category,total_customers,total_churned,churn_rate_percentage
0,High (70+),3583,1267,35.36
1,Medium (31-70),1807,440,24.35
2,Low (0-30),1653,162,9.80


------------------------------

